In [1]:
import sys
from time import sleep
import urllib.request as ur

from rich import print

sys.path.append('..')
from base import BaseCralwer, tryfindelements
from utils import parsingFile, getLogger, tm2int
from urls import COUPANG_LOGIN_URL, COUPANG_SEARCH_URL
from selenium.common.exceptions import NoSuchElementException

from selenium import webdriver

## Base

In [111]:
driver = webdriver.Chrome('/home/gunmo/public/chromedriver')
driver.implicitly_wait(1)

login_key = parsingFile('../config/login')
prod_key = parsingFile('../config/product')
findE_id = driver.find_element_by_id
findE_xp = driver.find_element_by_xpath
findE_lt = driver.find_element_by_link_text
findE_cn = driver.find_element_by_class_name
findE_cs = driver.find_element_by_css_selector

findEs_cn = driver.find_elements_by_class_name
findEs_cs = driver.find_elements_by_css_selector

## Login

In [112]:
driver.get(COUPANG_LOGIN_URL)

In [113]:
findE_id('login-email-input').send_keys(login_key['coupang']['id'])
findE_id('login-password-input').send_keys(login_key['coupang']['pw'])

In [114]:
findE_cn('login__button').click()

## search_products

In [129]:
from urllib import parse
url = parse.urlparse(COUPANG_SEARCH_URL.format(keyword)) 

In [130]:
query = parse.parse_qs(url.query)
parse.urlencode(query, doseq=True)

'q=%EB%8B%8C%ED%85%90%EB%8F%84+%EC%8A%A4%EC%9C%84%EC%B9%98&channel=user'

In [115]:
driver.get(COUPANG_SEARCH_URL.format(prod_key['product1']['keyword']))

In [116]:
driver.current_url

'https://www.coupang.com/np/search?component=&q=%EB%8B%8C%ED%85%90%EB%8F%84%20%EC%8A%A4%EC%9C%84%EC%B9%98&channel=user'

In [122]:
keyword = prod_key['product1']['keyword']
COUPANG_SEARCH_URL.format(keyword)

"https://www.coupang.com/np/search?component=&q=b'\\xb4\\xd1\\xc5\\xd9\\xb5\\xb5 \\xbd\\xba\\xc0\\xa7\\xc4\\xa1'&channel=user"

In [78]:
products = []
prices = []
links = []

In [ ]:
pages = 10
while pages:
    t_pro = findEs_cs("a > dl > dd > div > div.name")
    t_pri = findEs_cs("div.price > em > strong")
    t_links = findEs_cs("form > div > div > ul > li > a")
    
    for prod, price, link in zip(t_pro, t_pri, t_links):
        price = tm2int(price.text)
        if price > prod_key['product1']['high_price']: continue
        if price < prod_key['product1']['low_price']: continue
        products.append(prod.text)
        prices.append(price)
        links.append(link.get_attribute("href"))
    
    findE_cn("btn-next").click()
    pages -= 1

## check_products

### case 1: 구매 가능 상품

In [94]:
driver.get("https://www.coupang.com/vp/products/226814044?itemId=718780744&vendorItemId=4818819507&q=%EB%8B%8C%ED%85%90%EB%8F%84+%EC%8A%A4%EC%9C%84%EC%B9%98&itemsCount=36&searchId=ded2cdf81db24be1bfe251bcf319e11e&rank=377&isAddedCart=")

In [83]:
findEs_cn("prod-buy-btn")

[<selenium.webdriver.remote.webelement.WebElement (session="c5d0e9ad0613bcdf459672979208b80d", element="0967c621-2d09-45f0-a689-6c7cb55283b8")>]

In [95]:
findE_cn("prod-buy-btn").text

'바로구매'

### case 2: 구매 불가능 상품

In [96]:
driver.get("https://www.coupang.com/vp/products/1384804427?vendorItemId=70413795361&isAddedCart=")

In [85]:
findEs_cn("prod-buy-btn")

[<selenium.webdriver.remote.webelement.WebElement (session="c5d0e9ad0613bcdf459672979208b80d", element="7ba31094-108b-446f-a54a-777dafb18a2b")>]

In [97]:
findE_cn("prod-buy-btn").text

''